<a href="https://colab.research.google.com/github/shnhrtkyk/JTCcode/blob/main/01_%E5%9F%BA%E6%9C%AC%E6%93%8D%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 点群処理の基礎


## コードをダウンロード

In [ ]:
!git clone --recursive https://github.com/isl-org/Open3D

## Open3Dのインストール

In [ ]:
!pip install open3d==0.16

# 入出力

## 入力と表示

In [ ]:
!wget https://github.com/isl-org/open3d_downloads/releases/download/20220201-data/fragment.ply

In [ ]:
import sys
import numpy as np
import open3d as o3d

filename = "/content/fragment.ply"

print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)

print(pcd)
print(np.asarray(pcd.points))

pcds = []
pcds.append(o3d.io.read_point_cloud(filename))
o3d.visualization.draw_geometries([pcds[0]], zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])
o3d.visualization.draw_plotly(
  [pcds[0]],
  width=800,
  height=800,
  front=[0.4257, -0.2125, -0.8795],
  lookat=[2.6172, 2.0475, 1.532],
  up=[-0.0694, -0.9768, 0.2024]
)



# サンプリング

### 等間隔サンプリング

In [ ]:
print("Downsample the point cloud with a voxel of 0.05")
downpcd = pcd.voxel_down_sample(voxel_size=0.05)
o3d.visualization.draw_geometries([downpcd])
print(downpcd)
o3d.visualization.draw_plotly(
[downpcd],
width=1200,
height=800,
front=[0.4257, -0.2125, -0.8795],
                                      lookat=[2.6172, 2.0475, 1.532],
                                      up=[-0.0694, -0.9768, 0.2024]
)

### 最遠点サンプリング

In [ ]:
def l2_norm(a, b):
    return ((a - b) ** 2).sum(axis=1)

def farthest_point_sampling(pcd, k, metrics=l2_norm):
    indices = np.zeros(k, dtype=np.int32)
    points = np.asarray(pcd.points)
    distances = np.zeros((k, points.shape[0]), dtype=np.float32)
    indices[0] = np.random.randint(len(points))
    farthest_point = points[indices[0]]
    min_distances = metrics(farthest_point, points)
    distances[0, :] = min_distances
    for i in range(1, k):
        indices[i] = np.argmax(min_distances)
        farthest_point = points[indices[i]]
        distances[i, :] = metrics(farthest_point, points)
        min_distances = np.minimum(min_distances, distances[i, :])
    pcd = pcd.select_by_index(indices)
    return pcd

# main
filename = "/content/fragment.ply"
k = 2000
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

o3d.visualization.draw_geometries([pcd], zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])
o3d.visualization.draw_plotly(
[pcd],
width=1200,
height=800
)
downpcd = farthest_point_sampling(pcd, k)
print(downpcd)

o3d.visualization.draw_geometries([downpcd], zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])



o3d.visualization.draw_plotly(
[downpcd],
width=1200,
height=800,
front=[0.4257, -0.2125, -0.8795],
                                      lookat=[2.6172, 2.0475, 1.532],
                                      up=[-0.0694, -0.9768, 0.2024]
)

# ノイズ除去

## 統計的外れ値除去

In [ ]:
import sys
import open3d as o3d

def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                      zoom=0.3412,
                                      front=[0.4257, -0.2125, -0.8795],
                                      lookat=[2.6172, 2.0475, 1.532],
                                      up=[-0.0694, -0.9768, 0.2024])
    o3d.visualization.draw_plotly(
      [inlier_cloud, outlier_cloud],
      width=1200,
      height=800,
      front=[0.4257, -0.2125, -0.8795],
                                      lookat=[2.6172, 2.0475, 1.532],
                                      up=[-0.0694, -0.9768, 0.2024]
      )

filename = "/content/fragment.ply"
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

print("Statistical oulier removal")
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
display_inlier_outlier(pcd, ind)

print("Radius oulier removal")
cl, ind = pcd.remove_radius_outlier(nb_points=16, radius=0.02)
display_inlier_outlier(pcd, ind)


## 幾何的外れ値除去

In [ ]:
import sys
import open3d as o3d

def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                      zoom=0.3412,
                                      front=[0.4257, -0.2125, -0.8795],
                                      lookat=[2.6172, 2.0475, 1.532],
                                      up=[-0.0694, -0.9768, 0.2024])
    o3d.visualization.draw_plotly(
      [inlier_cloud, outlier_cloud],
      width=1200,
      height=800,
      front=[0.4257, -0.2125, -0.8795],
                                      lookat=[2.6172, 2.0475, 1.532],
                                      up=[-0.0694, -0.9768, 0.2024]
      )

filename = "/content/fragment.ply"
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

print("Radius oulier removal")
cl, ind = pcd.remove_radius_outlier(nb_points=100, radius=0.02)
display_inlier_outlier(pcd, ind)


# 法線推定

## 法線推定

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import plotly.graph_objects as go

# Estimation of normal vector of points
pcd.estimate_normals(
search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
points = np.asarray(pcd.points)
colors = (0.5, 0.5, 0.5) + np.asarray(pcd.normals) * 0.5

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points[:,0], y=points[:,1], z=points[:,2],
            mode='markers',
            marker=dict(size=1, color=colors)
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.show()